In [2]:
#Identification for Praw
import praw
reddit = praw.Reddit(
    client_id=
    client_secret=
    user_agent=
    username=
    password=
)

In [3]:
#Taking Top20 Post at AskReddit
subreddit = reddit.subreddit('AskReddit').top("day", limit=30)
top_20 = []
for submission in subreddit:
    if not submission.over_18:
        top_20.append(submission)
    if len(top_20) == 20:
        break
    #print(submission.over_18)
#for submission in subreddit.top("day", limit=20):
#    top_20.append(submission)
for i, top in enumerate(top_20):
    print(i+1, " : ", top.title)

1  :  Women of reddit, what do you think about the fall of constantinople in 1453 by the ottoman forces led by Mehmed II?
2  :  People who have worked at Goodwill or other thrift stores etc. and processed donations, what’s the craziest thing you’ve found?
3  :  Employers/Inteviewers of Reddit, why do you ask the "So why are you interested in this job?" to people interviewing for basic jobs like a pizza delivery driver or a receptionist?
4  :  What is the weirdest thing your body does that no one else’s does?
5  :  What's something good that happened in 2020?
6  :  You can beat a person up but the catch is you have do it in synchronization with music. What music are you going to choose?
7  :  In your opinion, what is the ugliest word in the English language?
8  :  What opinion do you have that you hesitate to express in real life?
9  :  What’s the worst mistake human(s) have made?
10  :  What is the stupidest thing you saw someone do in high school?
11  :  You find a gun in your mailbox

In [4]:
import pymysql
connection = pymysql.connect(host='',
                            user='',
                            password='',
                            db='',
                            charset='utf8mb4',
                            )

In [5]:
#데이터 전처리 및 깊이 탐색 부분
from praw.models import MoreComments
import crwaldef

In [1]:
oneday_max_depth = []
oneday_three_max_depth = []
for post in top_20[1:19]:

    all_list = []
    comment_info = []
    
    post_info = crwaldef.parse_submission(post)   #Post정보
    crwaldef.print_Data(post_info) #Post정보 출력
    
    post_comment = post.comments.replace_more(limit=None)  #Comments의 모든 정보를 사용할 수 있게 만든다.
    for comment in post.comments.list(): #정보들의 대표 값을 list에 삽입
        comment_info.append(crwaldef.parse_comment(comment))
    
    comment_length = len(comment_info) #스팸 데이터를 지운 후의 댓글 수
    #print('comment_count : ', comment_length, "\n")
    comment_info = crwaldef.comment_Preprocessing(comment_info)

    for comment in comment_info:
        crwaldef.all_Find_Depth(comment, all_list)
    all_list.sort(key=lambda x:x[2], reverse=True)

    final_list = crwaldef.tree_Clean(all_list)

    top_3_comments = []
    top_3_ids = []
    for i in final_list:
        top_c, top_i = in_top3(i[1])
        top_3_comments.append(top_c)
        top_3_ids.append(top_i)
        #top_3_comments.append(in_top3(i[1]))
    #print(top_3_comments)
    #print('코멘트 라인 수 : ', len(top_3_comments))
    #print('각 라인 길이 : ', len(top_3_comments[0]), " ", len(top_3_comments[1]), " ", len(top_3_comments[2]))
    oneday_max_depth.append(len(top_3_comments[0]))
    for i in range(len(top_3_comments)):
        oneday_three_max_depth.append(len(top_3_comments[i]))

    save_comments = crwaldef.jump_Comment(top_3_comments)
    #save_ids = crwaldef.jump_Comment(top_3_ids)
    #print(save_comments)
    #print(save_ids)
    cursor = connection.cursor()
    sql = "insert into new_conversation(parent_com, child_com) values(%s, %s)"
    #sql1 = "insert into id(parent_id, child_id) values(%s, %s)"
    cursor.executemany(sql, save_comments)
    #cursor.executemany(sql1, save_ids)


connection.commit()
connection.close()

NameError: name 'top_20' is not defined